In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
with open('/gdrive/My Drive/test.txt', 'w') as f:
    f.writelines("test")

In [ ]:
!pip install ratsnlp

In [ ]:
# 코포라를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리
# 데이터는 네이버 영화 리뷰 NSMC
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)

In [ ]:
# NSMC 전처리
# NSMC에 포함된 영화 리뷰를 순수 텍스트 형태로 저장
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.test.get_all_texts())

### GPT 토크나이저 구축
- 문자 단위가 아닌 유니코드 바이트 수준으로 어휘 집합을 구축하고 토큰화를 수행
- 바이트 수준으로 BPE를 수행한다는 것은 어휘 집합 구축 대상 말뭉치를 위와 같이 변환, 이것을 문자 취급해 가장 자주 ㄷㅇ장한 문자열을 병합하는 방식으로 어휘 집합 생성.

In [ ]:
# 디렉토리 생성
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok = True)

In [ ]:
# 바이트 수준 BPE 어휘 집합 구축
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

### BERT 토크나이저 구축
- BERT는 워드피스 토크나이저 사용

In [ ]:
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok = True)

In [ ]:
# 워드피스 어휘 집합 구축
from tokenizers import BertWordPieceTokenizer

wordpiece_tokenizer = BertWordPieceTokenizer()
wordpiece_tokenizer.train(
    files = ["/content/train.txt", "/content/test.txt"],
    vocab_size = 10000,
)

wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

### 토큰화

In [ ]:
# GPT 토크나이저 선언
from transformers import GPT2Tokenizer

tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

In [ ]:
# GPT 토크나이저로 토큰화 하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenizer_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [ ]:
tokenizer_sentences

In [ ]:
# GPT 모델 입력 만들기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [ ]:
# BERT 입력값 만들기
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case = False,
)

In [ ]:
# BERT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

tokenized_sentences

In [ ]:
# BERT 모델 입력 만들기
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [ ]:
batch_inputs['input_ids']